In [ ]:
# --- 1. Imports ---
import os
import re
from typing import Any, Dict, List, Tuple

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_recall_fscore_support,
)

from openai import OpenAI


# --- 2. File Path ---
csv_path = "/Users/stevefeng/Downloads/mentoring_ground_truth.csv"


# --- 3. Load CSV ---
df = pd.read_csv(csv_path)
print(f"✅ Loaded dataset with {len(df)} rows")
print("Columns:", list(df.columns))
print(df.head())


# --- 4. Identify Columns ---
# Adjust these names if your CSV has different ones
text_col_candidates = [c for c in df.columns if c.lower() in ("comment", "text", "body")]
label_col_candidates = [c for c in df.columns if c.lower() in ("label", "mentoring", "y")]

TEXT_COL = text_col_candidates[0] if text_col_candidates else df.columns[0]
LABEL_COL = label_col_candidates[0] if label_col_candidates else df.columns[1]


# --- 5. Normalize labels ---
def normalize_label(v):
    s = str(v).strip().upper()
    if s in {"Y", "YES", "1", "MENTORING", "0"}:
        return 0
    if s in {"N", "NO", "-1", "NOT", "NOT MENTORING"}:
        return -1
    try:
        iv = int(float(s))
        return 0 if iv == 0 else -1
    except Exception:
        return -1


df["label_norm"] = df[LABEL_COL].apply(normalize_label)


# 6.0 Columns: prefer an explicit "comments" column if present; else use detected TEXT_COL
PREFERRED_TEXT_COL = "comments"
if PREFERRED_TEXT_COL in df.columns:
    TEXT_COL = PREFERRED_TEXT_COL  # override to your desired column
# LABEL_COL already set to "label_norm" above

# Quick sanity checks
assert TEXT_COL in df.columns, f"❌ Text column '{TEXT_COL}' not found in df.columns"
assert "label_norm" in df.columns, "❌ normalized label 'label_norm' not found"
LABEL_COL = "label_norm"


# ============================================================
# FIXED preprocessing (this is the key)
# - Keep raw as-is
# - Strip quoted lines BEFORE flattening newlines
# - If stripping nukes everything, fall back to raw (so not empty)
# ============================================================

def strip_quoted_lines_preserve_newlines(text: str) -> str:
    if text is None:
        return ""
    lines = str(text).splitlines()
    kept = []
    for ln in lines:
        s = ln.strip()
        if not s:
            continue
        if s.startswith(">"):
            continue
        kept.append(ln)
    return "\n".join(kept).strip()

def normalize_one_line(text: str) -> str:
    if text is None:
        return ""
    text = str(text)
    text = re.sub(r"[\u200b\uFEFF\xa0]", " ", text)
    text = re.sub(r"\r\n|\r|\n", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def make_text_sent_to_model(text_raw: str) -> str:
    stripped = strip_quoted_lines_preserve_newlines(text_raw)
    sent = normalize_one_line(stripped)
    if sent.strip() == "":
        sent = normalize_one_line(text_raw)
    return sent


# Keep raw
df["text_raw"] = df[TEXT_COL].fillna("").astype(str)

# For display/debug: flattened raw
df["text_norm_full"] = df["text_raw"].apply(normalize_one_line)

# This is what the model sees (quote-stripped + flattened, with fallback)
df["text_sent_to_model"] = df["text_raw"].apply(make_text_sent_to_model)

# Debug how many became empty (should be much smaller now)
empty_after = (df["text_sent_to_model"].astype(str).str.strip() == "")
print(f"\n⚠️ Empty AFTER preprocessing (text_sent_to_model): {int(empty_after.sum())} / {len(df)}")
if empty_after.any():
    dbg = df.loc[empty_after, ["text_raw", "text_norm_full", "text_sent_to_model", "label_norm"]].copy()
    dbg.to_csv("empty_after_preprocess_debug.csv", index=False)
    print("📄 Saved: empty_after_preprocess_debug.csv")


# --- Rule-based short-text guard (kept) ---
def rule_based_neg(comment: str) -> int | None:
    if not comment:
        return None
    text = str(comment).strip()
    words = [w for w in re.findall(r"\b\w+\b", text)]
    if len(words) < 3:
        return -1
    return None


# --- OpenAI client + helpers (kept) ---
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ OPENAI_API_KEY not set.")
client = OpenAI(api_key=api_key)
MODEL = "gpt-4o-mini"


def to_label(resp: str) -> int:
    s = (resp or "").strip()
    s = s.replace("\u2013", "-").replace("\u2014", "-")
    m = re.search(r"-?\d+", s)
    if m:
        try:
            v = int(m.group(0))
            if v in (0, 1):
                return 0
            if v == -1:
                return -1
        except ValueError:
            pass
    if "mentoring" in s.lower() and "not mentoring" not in s.lower():
        return 0
    if "not mentoring" in s.lower():
        return -1
    if re.search(r"\b0\b", s):
        return 0
    return -1


def classify_once(prompt: str) -> tuple[int, str]:
    for attempt in range(2):
        r = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_MSG},
                {"role": "user", "content": prompt},
            ],
            max_completion_tokens=10,
        )
        raw = (r.choices[0].message.content or "").strip()
        if raw:
            return to_label(raw), raw
        prompt = prompt + "\nAnswer strictly with 0 or -1."
    return -1, ""


SYSTEM_MSG = """You are a precise classifier for GitHub code-review comments. Output ONLY a single token: 0 or -1. Never output 1.
Before classification:
1. Ignore any quoted text from previous messages (lines starting with '>').
2. Ignore sections that are clearly repeated from earlier in the same thread.
3. Only analyze the new, original content written by the commenter."""

BASE = """You are a GitHub code-review comment classifier.

Label each comment as:
- **Mentoring (0)** — when it *helps another contributor learn, reason, or improve their code or design.*
- **Not Mentoring (-1)** — when it is purely procedural, administrative, or social.

### Mentoring (0)
A comment is mentoring if it does ANY of the following:
• Explains *why* something should change, or the *reason* behind a design choice.  
• Suggests specific improvements, configurations, or refactorings.  
• Evaluates trade-offs (“better in central location because…”).  
• Asks targeted *technical* questions that help debugging or design understanding.  
• Provides examples, links, or suggestions.
• Provides resources, guides to other resources/similar issues.
• Asks questions to guide the author toward better understanding.
• Expresses agreement *with rationale* (“agree it should be fixed centrally” = mentoring).
• Guides on the project culture or best practices with explanations.
• Gives **operational or CI/CD guidance with a clear explanation or diagnosis** (e.g., “Please re-run Jenkins; it’s flaky under load”) — this counts as mentoring because it helps others understand project infrastructure behavior.
• Asks *informative or clarifying questions* that help the commenter or others understand the project’s timeline, versioning, or integration context (e.g., asking in which version a feature will appear to plan dependent work).
• Provides project maintenance or release-process guidance **with explanation or rationale** — for example, clarifying CI failures, explaining rebase requirements, or teaching how to write changelog entries.
• Provides constructive rejection or corrective feedback **with explanation or reasoning**, helping the contributor understand project rules, constraints, or next steps.
• Evaluates alternative technical options or sufficiency (“you can create another configuration, but the embedded one should be enough”) — reasoning about trade-offs or adequacy counts as mentoring because it helps others understand project infrastructure behavior.
• Asks *informative or clarifying questions* that help the commenter or others understand the project’s timeline, versioning, or integration context (e.g., asking in which version a feature will appear to plan dependent work).
• Provides project maintenance or release-process guidance **with explanation or rationale** — for example, clarifying CI failures, explaining rebase requirements, or teaching how to write changelog entries.
• Provides constructive rejection or corrective feedback **with explanation or reasoning**, helping the contributor understand project rules, constraints, or next steps.
• Evaluates alternative technical options or sufficiency (“you can create another configuration, but the embedded one should be enough”) — reasoning about trade-offs or adequacy counts as mentoring.


### Not Mentoring (-1)
A comment is not mentoring if it is ONLY:
• Acknowledgement or courtesy (“LGTM”, “+1”).  
• Bot messages or automated responses.
• Simple release-schedule or availability inquiries without technical context (“When will this be merged?”, “Any ETA?”, “Which version?”) are not mentoring.
• Provides follow-up or maintenance instructions (e.g., adding markers, updating docs, or writing unit tests) and explains what to do or how to proceed — even when combined with appreciation or coordination.

Output exactly one number: **0** or **-1**, nothing else."""


# --- Curated few-shots (POS from your misclassified set + crisp NEG admin cases) ---

POS_CUES = [
    # reasoning / trade-off (existing)
    ("We should keep this fix in a central location so it's easier to maintain.", 0),
    # coordination but with rationale (existing)
    ("Let's apply this to both 4.7 and 4.8 so they stay consistent.", 0),
    # concise justification (existing)
    ("We cannot change the JSON format because other clients depend on it.", 0),
    # Process Teaching/RAG (NEW)
    ("Actually, we use labels to route PRs to the right sub-team for review. I applied one and reached out to the team to confirm.", 0),
    # Constructive Rejection (NEW)
    ("I'm going to close this PR for now since there are outstanding comments, just to get this off our PR review queue. Please re-open if you address the comments.", 0),
    # Setup/Maintenance Guidance (NEW)
    ("I suspect that this change requires me to update tests in the below files. Let me know if I need to change it or make any other changes.", 0),
]

NEAR_MISS_NEGS = [
    # Simple admin/social (existing)
    ("Thanks for the PR! I'll look into merging it. What's the 3rd point?", -1),
    ("OK I am +1 for this then.", -1),
    ("Please rebase after #1515 merges.", -1),
    # Pure CI/Status Update (NEW)
    ("I checked the Google testing failures, they looked like flakes unrelated to your change. Rerunning", -1),
    # Request for Admin Explanation (NEW)
    ("is there a simple explanation you can add here that does not require you to dig into the code?", -1),
    # Administrative Rebase/Force (NEW)
    ("I hope it's okay to just re-apply this commit onto upstream master using cherry-pick and force-push the branch.", -1),
]

def fewshot_block(examples):
    return "\n\n".join([f"Comment: {t}\nLabel: {y}" for t, y in examples])

FEWSHOT_CURATED = fewshot_block(POS_CUES + NEAR_MISS_NEGS)

def fewshot_block(examples):
    return "\n\n".join([f"Comment: {t}\nLabel: {y}" for t, y in examples])

FEWSHOT_CURATED = fewshot_block(POS_CUES + NEAR_MISS_NEGS)

def fs_strict(comment: str) -> str:
    return f"""{BASE}

Few-shot examples:
{FEWSHOT_CURATED}

Comment to classify:
{comment}
"""


# ============================================================
# Row-safe evaluation (prints FULL raw comments if misclassified)
# ============================================================
def eval_on_split(
    name: str,
    X_split: pd.Series,
    y_split: pd.Series,
    RAW_split: pd.Series,
    NORM_full_split: pd.Series,
    use_rules: bool = True,
) -> Dict[str, Any]:
    preds: List[int] = []
    rules: List[bool] = []
    raws: List[str] = []
    prompts_preview: List[str] = []
    detailed_rows: List[Dict[str, Any]] = []

    print(f"\n### Per-instance outputs for prompt: {name} | N={len(X_split)}")

    for i, (txt_sent, gold, txt_raw, txt_norm_full) in enumerate(zip(X_split, y_split, RAW_split, NORM_full_split)):

        # Build prompt
        prompt = fs_strict(txt_sent)
        prompt_preview = prompt[:240].replace("\n", " ") + ("..." if len(prompt) > 240 else "")

        # Guard: if empty after preprocessing, classify as -1 but DO NOT hide it
        if str(txt_sent).strip() == "":
            pred, raw_out = -1, "[SKIPPED_EMPTY_AFTER_PREPROCESS]"
            rules.append(True)
        else:
            if use_rules:
                rb = rule_based_neg(txt_sent)
                if rb is not None:
                    pred, raw_out = rb, "[RULE: short-text <3 words]"
                    rules.append(True)
                else:
                    pred, raw_out = classify_once(prompt)
                    rules.append(False)
            else:
                pred, raw_out = classify_once(prompt)
                rules.append(False)

        preds.append(int(pred))
        raws.append(raw_out)
        prompts_preview.append(prompt_preview)

        correct = (int(pred) == int(gold))

        print(f"\n[{i}] GOLD={int(gold)} | PRED={int(pred)} | {'✅' if correct else '❌'}")
        print("RAW COMMENT (FULL):")
        print(txt_raw)
        print("\nNORM FULL (FULL):")
        print(txt_norm_full)
        print("\nTEXT SENT TO MODEL (FULL):")
        print(txt_sent)
        print("\nRAW MODEL OUTPUT:")
        print(repr(raw_out))
        print("\nPROMPT_PREVIEW:")
        print(prompt_preview)

        detailed_rows.append({
            "index": i,
            "comment_sent_to_model": txt_sent,
            "comment_raw_full": txt_raw,
            "comment_norm_full": txt_norm_full,
            "gold_label": int(gold),
            "pred_label": int(pred),
            "correct": bool(correct),
            "raw_model_output": raw_out,
            "prompt_preview": prompt_preview,
            "rule_based": bool(rules[-1]),
        })

    preds_arr = np.array(preds, dtype=int)
    y_true = y_split.astype(int).values

    detailed_df = pd.DataFrame(detailed_rows)
    return {
        "name": name,
        "accuracy": float(accuracy_score(y_true, preds_arr)),
        "f1_macro": float(f1_score(y_true, preds_arr, average="macro")),
        "f1_weighted": float(f1_score(y_true, preds_arr, average="weighted")),
        "precision_macro": float(precision_recall_fscore_support(y_true, preds_arr, average="macro", zero_division=0)[0]),
        "recall_macro": float(precision_recall_fscore_support(y_true, preds_arr, average="macro", zero_division=0)[1]),
        "report": classification_report(y_true, preds_arr, digits=3, zero_division=0),
        "cm": confusion_matrix(y_true, preds_arr),
        "preds": preds_arr,
        "rule_hits": rules,
        "raws": raws,
        "prompt_previews": prompts_preview,
        "detailed_df": detailed_df,
    }
# --- Build features/labels from the full dataset ---
X_all = df["text_sent_to_model"].reset_index(drop=True)
RAW_all = df["text_raw"].reset_index(drop=True)
NORM_full_all = df["text_norm_full"].reset_index(drop=True)
y_all = df[LABEL_COL].astype(int).reset_index(drop=True)

N = len(X_all)
print(f"\n=== 10-fold CV on FULL DATASET ===")
print(f"N={N} | TEXT_COL='{TEXT_COL}' | LABEL_COL='{LABEL_COL}'")

# --- 10-fold stratified CV ---
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

fold_metrics: List[Dict[str, Any]] = []
all_detailed: List[pd.DataFrame] = []

for fold_idx, (_, test_idx) in enumerate(skf.split(X_all, y_all), start=1):
    print(f"\n================= Fold {fold_idx}/10 =================")
    X_test = X_all.iloc[test_idx].reset_index(drop=True)
    RAW_test = RAW_all.iloc[test_idx].reset_index(drop=True)
    NORM_full_test = NORM_full_all.iloc[test_idx].reset_index(drop=True)
    y_test = y_all.iloc[test_idx].reset_index(drop=True)

    res = eval_on_split(
        name=f"few_shot_strict_fold{fold_idx}",
        X_split=X_test,
        y_split=y_test,
        RAW_split=RAW_test,
        NORM_full_split=NORM_full_test,
        use_rules=False,
    )

    fold_df = res["detailed_df"]
    fold_detailed_path = f"mentoring_cv_fold{fold_idx}_detailed_outputs.csv"
    fold_df.to_csv(fold_detailed_path, index=False)
    print(f"\n📄 Saved per-instance details: {fold_detailed_path}")

    fold_mis_df = fold_df[fold_df["correct"] == False].copy()
    fold_mis_path = f"mentoring_cv_fold{fold_idx}_misclassified.csv"
    fold_mis_df.to_csv(fold_mis_path, index=False)
    print(f"📄 Saved misclassified: {fold_mis_path} | N={len(fold_mis_df)}")

    print(f"\n--- Fold {fold_idx} report ---")
    print(res["report"])
    print("Confusion matrix (rows=true [-1,0], cols=pred):\n", res["cm"])
    print(
        f"Acc: {res['accuracy']:.4f} | "
        f"F1-macro: {res['f1_macro']:.4f} | "
        f"Prec-macro: {res['precision_macro']:.4f} | "
        f"Recall-macro: {res['recall_macro']:.4f}"
    )

    fold_metrics.append({
        "fold": fold_idx,
        "accuracy": res["accuracy"],
        "f1_macro": res["f1_macro"],
        "precision_macro": res["precision_macro"],
        "recall_macro": res["recall_macro"],
    })

    all_detailed.append(fold_df)

# --- Aggregate CV metrics ---
cv_df = pd.DataFrame(fold_metrics)
cv_df.to_csv("mentoring_cv_10fold_metrics.csv", index=False)
print("\n📄 Saved CV metrics to mentoring_cv_10fold_metrics.csv")

def mean_std(series: pd.Series) -> Tuple[float, float]:
    mu = float(series.mean())
    sd = float(series.std(ddof=1)) if len(series) > 1 else 0.0
    return mu, sd

acc_mu, acc_sd = mean_std(cv_df["accuracy"])
f1_mu, f1_sd = mean_std(cv_df["f1_macro"])
pr_mu, pr_sd = mean_std(cv_df["precision_macro"])
rc_mu, rc_sd = mean_std(cv_df["recall_macro"])

print("\n================= 10-fold CV SUMMARY =================")
print(cv_df.to_string(index=False))
print(
    f"\nMEAN ± STD over 10 folds:\n"
    f"- Accuracy      : {acc_mu:.4f} ± {acc_sd:.4f}\n"
    f"- F1 (macro)    : {f1_mu:.4f} ± {f1_sd:.4f}\n"
    f"- Precision (macro): {pr_mu:.4f} ± {pr_sd:.4f}\n"
    f"- Recall (macro): {rc_mu:.4f} ± {rc_sd:.4f}"
)

all_detailed_df = pd.concat(all_detailed, ignore_index=True)
all_detailed_df.to_csv("mentoring_cv_ALL_detailed_outputs.csv", index=False)
print("\n📄 Saved ALL per-instance details: mentoring_cv_ALL_detailed_outputs.csv")

all_mis_df = all_detailed_df[all_detailed_df["correct"] == False].copy()
all_mis_df.to_csv("mentoring_cv_ALL_misclassified.csv", index=False)
print(f"📄 Saved ALL misclassified: mentoring_cv_ALL_misclassified.csv | N={len(all_mis_df)}")



✅ Loaded dataset with 100 rows
Columns: ['body', 'mentoring']
                                                body mentoring
0  Regarding the precise values used in the `_vs2...         n
1  Huh. Weird. Git says that upstream `master` do...         n
2  I checked the Google testing failures, they lo...         n
3                                                  🎉         n
4  please i just downloaded visual studio 2026 co...         n

⚠️ Empty AFTER preprocessing (text_sent_to_model): 0 / 100

=== 10-fold CV on FULL DATASET ===
N=100 | TEXT_COL='body' | LABEL_COL='label_norm'

================= Fold 1/10 =================

### Per-instance outputs for prompt: few_shot_strict_fold1 | N=10

[0] GOLD=0 | PRED=0 | ✅
RAW COMMENT (FULL):
In regard to the color situation, I think the current approach may not be ideal. The existing customer would have to set the background color from color scheme to preserve the existing behavior. We should avoid making the code to be more verbose if possibl